In [1]:
from TCSPC import *

In [2]:
EGFP = Simulation([0.497,0.503],[2.43,3.07])

In [ ]:
# tdata = np.loadtxt('EGFPt.csv')
# ydata = np.loadtxt('EGFPy2.csv')

In [55]:
def trim_rescale_data(tdata,ydata,end =int(0.6*380),rescale = True):
    max_idx = np.argmax(ydata) #index of data point with maximum photon count N(0)
    tdata = tdata[:end-max_idx] #start from t = 0
    ydata = ydata[max_idx:end]  #start from max.
    if rescale == True:
        yerr = ydata/ydata[0]*np.sqrt(1/ydata+1/ydata[0]) #error after scaling
        ydata = ydata/ydata[0] # scale y data such that the beginning is 1 
    else:
        yerr = np.sqrt(ydata)
    weights = 1/yerr #weighted by 1/yerr, yerr is error after scaling ydata
    return tdata,ydata,weights

In [5]:
def exp(t, A, tau):
    return A * np.exp(-t/tau)

In [33]:
tdata,ydata,weights = trim_rescale_data(EGFP.t,EGFP.y2)

In [88]:
def poisson_deviance_residual(observed, expected):
    # Ensure no invalid values
    # valid = (observed > 0) & (expected > 0)
    # residual = np.zeros_like(observed)
    # residual[valid] =  2 *  (observed[valid] * np.log(observed[valid] / expected[valid]) - (observed[valid] - expected[valid]))
    residual=  np.sqrt(abs(2 *  (observed* np.log(observed/ expected) - (observed- expected)))) #residual array
    return residual

def residual(p, t, data):
    v = p.valuesdict()
    generative = v['c'] #constant background
    M = 1
    while f'A{M}' in v:
        generative += exp(t, v[f'A{M}'], v[f'tau{M}'])
        M += 1
    
    return poisson_deviance_residual(data, generative) #lmfit.minimizer minimize the residual array in the sum of squared sense


In [89]:
def initial_params(M,A_guess,tau_guess,rescale = True):
    p = lmfit.Parameters()
    p.add_many(('c', 0, True, 0, 1)) #constant background
    for i in range(1,M+1): #for each component
        p.add_many((f'A{i}', A_guess[i-1], True,0), #amplitude
                   (f'tau{i}', tau_guess[i-1], True, 0)) #lifetime
    if rescale == True:
        p[f'A{M}'].set(expr = f'1 {"".join([f"- A{i}" for i in range(1,M)])}') #fix the amplitude of last component
    return p



### bi-exp decay fit to EGFP

In [99]:
EGFP.n_photon = int(1e4)
EGFP.multi_exp_data()
tdata,ydata,weights = trim_rescale_data(EGFP.t,EGFP.y2,end = int(0.5*380))
p1 = initial_params(2,EGFP.amp,EGFP.tau)
#p1['c'].set(value = 0, vary = False)
mi1 = lmfit.minimize(residual, p1, args=(tdata, ydata), method='powell')

print(lmfit.fit_report(mi1))


[[Fit Statistics]]
    # fitting method   = Powell
    # function evals   = 1105
    # data points      = 180
    # variables        = 4
    chi-square         = 0.72119082
    reduced chi-square = 0.00409768
    Akaike info crit   = -985.565506
    Bayesian info crit = -972.793679
[[Variables]]
    c:     0.00191120 +/- 0.00456714 (238.97%) (init = 0)
    A1:    0.12299559 +/- 0.01372984 (11.16%) (init = 0.497)
    tau1:  0.01057423 +/- 0.01658326 (156.83%) (init = 2.43)
    A2:    0.87700441 +/- 0.01372984 (1.57%) == '1 - A1'
    tau2:  2.74844446 +/- 0.07968688 (2.90%) (init = 3.07)
[[Correlations]] (unreported correlations are < 0.100)
    C(c, tau2)  = -0.899
    C(A1, tau2) = 0.588
    C(c, A1)    = -0.301


In [98]:
result = mi1
del result.params['A2']

In [22]:
info_df,par_df = fit_df([result])


In [23]:
par_df

c  \
0 _val                                                 0.000854   
  init_value                                                  0   
  stderr                                               0.000457   
  correl      {'A0': '-0.734', 'tau0': '-0.701', 'tau1': '-0...   

                                                             A0  \
0 _val                                                  0.50092   
  init_value                                              0.497   
  stderr                                               0.842672   
  correl      {'c': '-0.734', 'tau0': '0.998', 'tau1': '0.998'}   

                                                         tau0  \
0 _val                                               2.471439   
  init_value                                             2.43   
  stderr                                             0.429631   
  correl      {'c': '-0.701', 'A0': '0.998', 'tau1': '0.994'}   

                                                         tau1  
0 _val                                               3.030307  
  init_value                                             3.07  
  stderr                                             0.526496  
  correl      {'c': '-0.769', 'A0': '0.998', 'tau0': '0.994'}

No rescale

In [82]:
EGFP.n_photon = int(1e3)
EGFP.multi_exp_data()
tdata,ydata,weights = trim_rescale_data(EGFP.t,EGFP.y2,end = 100,rescale = False)
p1 = initial_params(2,[np.max(EGFP.y2*0.48),np.max(EGFP.y2)*0.52],EGFP.tau,rescale = False)
#p1['c'].set(value = 0, vary = False)
mi1 = lmfit.minimize(residual, p1, args=(tdata, ydata),method = 'powell')

print(lmfit.fit_report(mi1))



[[Fit Statistics]]
    # fitting method   = Powell
    # function evals   = 186
    # data points      = 90
    # variables        = 5
    chi-square         = 82.4485887
    reduced chi-square = 0.96998340
    Akaike info crit   = 2.11287354
    Bayesian info crit = 14.6119219
##  Warning: uncertainties could not be estimated:
    c:     at boundary
[[Variables]]
    c:     1.1102e-16 +/- 3.0820e-08 (27760419948.92%) (init = 0)
    A0:    2.62888624 +/- 9.21073992 (350.37%) (init = 13.92)
    tau0:  2.23211847 +/- 1.72816291 (77.42%) (init = 2.43)
    A1:    15.2704387 +/- 9.70356586 (63.54%) (init = 15.08)
    tau1:  3.00826193 +/-        nan (nan%) (init = 3.07)
[[Correlations]] (unreported correlations are < 0.100)
    C(A0, tau0) = 2.811
    C(tau0, A1) = -2.665
    C(A0, A1)   = -0.996


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lmfit/minimizer.py:854: RuntimeWarning: invalid value encountered in sqrt
  (par.stderr * np.sqrt(self.result.covar[jvar, jvar])))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lmfit/minimizer.py:847: RuntimeWarning: invalid value encountered in sqrt
  par.stderr = np.sqrt(self.result.covar[ivar, ivar])


In [67]:
mi1.residual

array([3.23010367e-01, 1.11780578e+00, 1.08012935e+00, 9.23148275e-01,
       7.84991093e-01, 4.46317159e-01, 1.49981781e+00, 1.37362776e+00,
       2.31983165e-01, 9.71336235e-01, 1.45908176e+00, 1.31658111e+00,
       5.84747640e-02, 1.00239769e+00, 2.12514953e-03, 3.23141413e-01,
       5.58006134e-01, 8.65667968e-01, 7.00225782e-02, 2.08446431e-01,
       1.01564761e+00, 1.83665333e+00, 1.56390154e-01, 5.53538717e-01,
       5.88202171e-02, 5.26671002e-01, 8.31454940e-01, 2.89248550e-01,
       1.15748512e+00, 1.18024379e+00, 1.47600953e+00, 8.14355380e-01,
       1.69151823e+00, 3.16307852e-01, 6.26593688e-01, 1.40670567e+00,
       2.75317275e-01, 8.78227862e-01, 1.20239279e-01, 8.83640773e-02,
       3.77779324e-01, 1.21513208e-01, 5.98685204e-01, 6.55437871e-01,
       2.44730539e-02, 3.74655865e-01, 6.16500875e-01, 2.13010864e-01,
       1.66291299e-01, 5.88354178e-01, 1.06905743e+00, 3.22037914e-01,
       7.55118671e-01, 1.00235466e+00, 1.16161143e+00, 4.21507649e-01,
      

### mono-exp decay fit to EGFP

In [109]:
p1

name,value,initial value,min,max,vary,expression
c,0.00000000,0,0.00000000,1.00000000,True,
A0,0.49700000,0.497,0.00000000,inf,True,
tau0,2.43000000,2.43,0.00000000,inf,True,
A1,0.50300000,0.503,0.00000000,inf,False,1 - A0
tau1,3.07000000,3.07,0.00000000,inf,True,


In [110]:
p1 = initial_params(1,EGFP.amp,EGFP.tau)
mi1 = lmfit.minimize(residual, p1, args=(tdata, ydata),method = 'differential evaluation')

print(lmfit.fit_report(mi1))


[[Fit Statistics]]
    # fitting method   = Nelder-Mead
    # function evals   = 54
    # data points      = 1
    # variables        = 2
    chi-square         = 2.7040e-05
    reduced chi-square = 2.7040e-05
    Akaike info crit   = -6.51819185
    Bayesian info crit = -10.5181918
##  Warning: uncertainties could not be estimated:
[[Variables]]
    c:     0.00254108 (init = 0)
    A0:    1.00000000 == '1 '
    tau0:  2.72300093 (init = 2.43)


In [118]:
np.sum(residual(mi1.params,tdata,ydata))

29.50655967458046